In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import fiona

pd.options.display.max_columns = None 

# set working directory
os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

In [4]:
# Load Data
spills = gpd.read_file('flowlines_with_spills.geojson')
no_spills = gpd.read_file('flowlines_without_spills.geojson')

# Data Cleaning


## No Spills

In [5]:
print(no_spills.info())
print(no_spills.shape)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14853 entries, 0 to 14852
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ACTIONDESCRIPTION  2807 non-null   object        
 1   BEDDINGMATERIAL    7683 non-null   object        
 2   COMPANY_NAME       14853 non-null  object        
 3   CONSTRUCTDATE      14853 non-null  datetime64[ms]
 4   Diam_in            14853 non-null  float64       
 5   ENDLAT             14853 non-null  float64       
 6   ENDLONG            14853 non-null  float64       
 7   ENTIRELINEREMOVED  1765 non-null   object        
 8   FLOWLINEACTION     10497 non-null  object        
 9   FLOWLINEID         14842 non-null  float64       
 10  Fluid              14853 non-null  object        
 11  LOCATIONTYPE       14853 non-null  object        
 12  LOCATION_ID        14841 non-null  float64       
 13  Length_ft          14853 non-null  float64       
 14

In [6]:
no_spills.describe()

,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,FLOWLINEID,LOCATION_ID,Length_ft,MAXOPPRESSURE,OPERATOR_NUM,RECEIVE_DATE,SHAPE_Length,STARTLAT,STARTLOCATIONID,STARTLONG
count,14853,14853.000000,14853.000000,14853.000000,14842.000000,14841.000000,14853.000000,6086.000000,14853.000000,14853,14853.000000,14853.000000,14562.000000,14853.000000
mean,2000-08-18 20:07:13.366000,3.034890,39.976417,-104.574983,469334.193842,405605.562765,4919.429999,467.190766,36935.902713,2020-04-11 04:18:53.304000,1499.991087,39.976332,348049.554182,-104.575305
min,1900-01-11 00:00:00,0.000000,37.010428,-109.045542,455152.000000,159601.000000,2.200000,0.000000,710.000000,2018-04-25 14:38:45.767000,0.671472,36.993609,159652.000000,-109.049983
25%,1992-01-01 00:00:00,2.000000,39.999330,-104.920760,464616.250000,328266.000000,664.170000,40.000000,10633.000000,2019-07-05 15:17:12.473000,202.925598,39.998776,317506.000000,-104.926462
50%,2005-12-09 00:00:00,2.000000,40.157850,-104.683675,469103.000000,434300.000000,1295.080000,150.000000,10699.000000,2019-10-25 12:27:07.133000,395.337163,40.158210,328918.000000,-104.685410
75%,2011-01-23 00:00:00,3.000000,40.357490,-104.226813,474936.500000,464139.000000,2264.010000,500.000000,47120.000000,2020-11-20 08:28:58.403000,692.400139,40.357510,338344.000000,-104.227525
max,2020-06-01 00:00:00,16.000000,40.992630,-102.045863,484080.000000,483552.000000,152023.900000,4700.000000,200077.000000,2023-11-17 11:11:43.017000,46342.379790,40.996000,484366.000000,-102.046467
std,NaN,2.201265,0.822864,1.314217,6439.701624,66796.398523,19078.810361,817.984270,35550.631440,NaN,5814.379003,0.822716,50341.512489,1.314153


### Get line age from construction date

In [7]:
# Verify CONSTRUCTDATE is datetime type; convert if necessary
no_spills['CONSTRUCTDATE'] = pd.to_datetime(no_spills['CONSTRUCTDATE'])

# Calculate line_age
# Today's date
today = pd.Timestamp.now()

# Calculate the difference in years
no_spills['line_age_yr'] = (today - no_spills['CONSTRUCTDATE']).dt.days / 365.25

# Display the updated GeoDataFrame to verify 'line_age' column
print(no_spills[['CONSTRUCTDATE', 'line_age_yr']])

      CONSTRUCTDATE  line_age_yr
0        1983-11-09    40.840520
1        1983-12-07    40.763860
2        2006-06-05    18.269678
3        2007-04-08    17.429158
4        2005-05-10    19.340178
...             ...          ...
14848    2001-10-02    22.943190
14849    2008-02-22    16.553046
14850    2007-08-05    17.103354
14851    1998-09-27    25.957563
14852    1983-10-21    40.892539

[14853 rows x 2 columns]


In [8]:
# Print the list of column names
column_names = no_spills.columns.tolist()

print(column_names)

['ACTIONDESCRIPTION', 'BEDDINGMATERIAL', 'COMPANY_NAME', 'CONSTRUCTDATE', 'Diam_in', 'ENDLAT', 'ENDLONG', 'ENTIRELINEREMOVED', 'FLOWLINEACTION', 'FLOWLINEID', 'Fluid', 'LOCATIONTYPE', 'LOCATION_ID', 'Length_ft', 'MAXOPPRESSURE', 'Material', 'OPERATOR_NUM', 'Operator', 'PIPEMATERIAL', 'RECEIVE_DATE', 'SHAPE_Length', 'STARTLAT', 'STARTLOCATIONID', 'STARTLONG', 'Status', 'TYPEOFFLUIDTRANS', 'geometry', 'line_age_yr']


In [9]:
mapping = {
    'KINDER MORGAN CO2 CO LP': 'KINDER MORGAN CO2 CO LLC',
    'BEEMAN OIL & GAS INC': 'BEEMAN OIL & GAS LLC',
}
no_spills['Operator'] = no_spills['Operator'].replace(mapping)

no_spills.rename(columns={'OPERATOR_NUM': 'operator_number'}, inplace=True)
no_spills.rename(columns={'Operator': 'operator_name'}, inplace=True)

# Extract unique OPERATOR_NUM values
unique_operator_nums = no_spills['operator_number'].unique()

# Create a DataFrame to map OPERATOR_NUM to Operator (assuming the column name for operator names is 'Operator')
no_spills_operator_mapping = no_spills[['operator_number', 'operator_name']].drop_duplicates().reset_index(drop=True)

# Display the mapping
print(no_spills_operator_mapping)

     operator_number                          operator_name
0              10633  CRESTONE PEAK RESOURCES OPERATING LLC
1              68710          PETERSON ENERGY OPERATING INC
2              10459               EXTRACTION OIL & GAS INC
3              10646                   AXIS EXPLORATION LLC
4              10575                            8 NORTH LLC
..               ...                                    ...
114            47120        KERR MCGEE OIL & GAS ONSHORE LP
115            61250              MULL DRILLING COMPANY INC
116           100264                         XTO ENERGY INC
117            10000          BP AMERICA PRODUCTION COMPANY
118            10447             URSA OPERATING COMPANY LLC

[119 rows x 2 columns]


In [10]:
# List of columns to be removed
columns_to_remove = [
    "BEDDINGMATERIAL", "COMPANY_NAME", "ENDLAT", "ENDLONG", "ENTIRELINEREMOVED",
    "RECEIVE_DATE", "STARTLAT", "STARTLOCATIONID", "STARTLONG","ACTIONDESCRIPTION","operator_name",
    "TYPEOFFLUIDTRANS", "PIPEMATERIAL", "CONSTRUCTDATE"
]

# Drop the columns from the DataFrame
no_spills = no_spills.drop(columns=columns_to_remove)

In [11]:
# drop any NAs
no_spills = no_spills.dropna()

In [12]:
# Reorder df
new_order = ['operator_number', 'FLOWLINEID', 'LOCATION_ID', 'Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 
             'Fluid', 'Material', 'Diam_in', 'Length_ft', 'MAXOPPRESSURE', 'SHAPE_Length', 'line_age_yr', 'geometry'] 
no_spills = no_spills[new_order]
no_spills

,operator_number,FLOWLINEID,LOCATION_ID,Status,FLOWLINEACTION,LOCATIONTYPE,Fluid,Material,Diam_in,Length_ft,MAXOPPRESSURE,SHAPE_Length,line_age_yr,geometry
1,10633,470445.0,470443.0,Active,Out of Service,Production Facilities,Multiphase,Carbon Steel,2.00,1025.98,250.0,312.594254,40.763860,"MULTILINESTRING ((507681.553 4440214.644, 5076..."
5,68710,467411.0,318620.0,Active,Pre-Abandonment Notice,Production Facilities,Crude Oil Emulsion,Fiberglass,2.00,276.03,150.0,84.099157,45.429158,"MULTILINESTRING ((497028.859 4434818.938, 4970..."
7,10459,462601.0,452637.0,Out of Service,Out of Service,Production Facilities,Multiphase,Carbon Steel,2.25,510.01,1140.0,155.389195,17.267625,"MULTILINESTRING ((496682.46 4457399.772, 49668..."
8,10459,462602.0,452637.0,Out of Service,Out of Service,Production Facilities,Multiphase,Carbon Steel,2.25,1597.85,1140.0,486.830336,14.351814,"MULTILINESTRING ((496340.456 4457384.126, 4963..."
9,10633,473671.0,336437.0,Abandoned,Out of Service,Production Facilities,Multiphase,Steel,2.00,3457.32,265.0,1053.369404,21.177276,"MULTILINESTRING ((507139.811 4448160.801, 5071..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14724,96155,456386.0,433999.0,Active,Registration,Production Facilities,Multi-Phase,Carbon Steel,3.50,1404.32,150.0,427.915127,6.247775,"MULTILINESTRING ((597095.529 4518130.982, 5970..."
14725,96155,456381.0,433999.0,Active,Registration,Production Facilities,Multi-Phase,Carbon Steel,3.50,1404.32,150.0,427.915127,6.239562,"MULTILINESTRING ((597095.529 4518130.982, 5970..."
14726,96155,456382.0,433999.0,Active,Registration,Production Facilities,Multi-Phase,Carbon Steel,3.50,1404.32,150.0,427.915127,6.247775,"MULTILINESTRING ((597095.529 4518130.982, 5970..."
14832,35080,455592.0,443145.0,Active,Registration,Production Facilities,Oil,HDPE,3.50,1175.83,40.0,358.332547,6.822724,"MULTILINESTRING ((637207.682 4380630.737, 6371..."


### Consolidate variables uniformly

In [13]:
# List of specific columns for which to print unique values
columns_to_check = ['Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid', 'Material']

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through the specified columns and store unique values in the dictionary
for column in columns_to_check:
    if column in no_spills.columns:  # Check if the column exists in the DataFrame
        unique_values = no_spills[column].unique().tolist()  # Convert numpy array to list
        unique_values_dict[column] = unique_values
    else:
        unique_values_dict[column] = "Column not found in DataFrame."

# Print each column's unique values list
for column, values in unique_values_dict.items():
    print(f"Unique values in {column}: {values}")

Unique values in Status: ['Active', 'Out of Service', 'Abandoned', 'ACTIVE', 'Inactive', 'abandoned', 'InActive', 'OOS', 'Abandoned in Place', 'ABANDONED', 'Actove', 'OutofService', 'Avtive', 'PA', 'INACTIVE', 'Out-of-Service', 'Shut in', 'Status', 'Future', 'Actve']
Unique values in FLOWLINEACTION: ['Out of Service', 'Pre-Abandonment Notice', 'Abandonment Verification', 'Realignment', 'Registration', 'Removed From Service', 'Abandonment']
Unique values in LOCATIONTYPE: ['Production Facilities', 'Well Site', 'Manifold', 'Compressor Station', 'Gathering Line', 'Crude Oil Transfer Line', 'Produced Water Transfer System']
Unique values in Fluid: ['Multiphase', 'Crude Oil Emulsion', 'Emulsion', 'NATUAL GAS', 'CO2/Produced Water', 'Oil', 'Co2/Prod Water', 'Natural Gas', 'Gas', 'Produced Water', 'MULTIPHASE', 'CRUDE OIL EMULSION', 'Water', 'Condensate', 'Crude Oil', 'Unprocessed Production Fluids', 'NATURAL GAS', 'Multi-Phase', 'CO2/Prod Water', 'CO2Produced Water', 'CO2/Produced Wtaer', 'C0

In [14]:
status_mapping = {
    'Active': 'Active', 'ACTIVE': 'Active', 'Actove': 'Active', 'Avtive': 'Active', 'Actve': 'Active',
    'Out of Service': 'Out of Service', 'OOS': 'Out of Service', 'OutofService': 'Out of Service', 'Out-of-Service': 'Out of Service',
    'Abandoned': 'Abandoned', 'abandoned': 'Abandoned', 'Abandoned in Place': 'Abandoned', 'ABANDONED': 'Abandoned',
    'Inactive': 'Inactive', 'InActive': 'Inactive', 'INACTIVE': 'Inactive',
    'PA': 'Pending Analysis', 'Shut in': 'Shut In',
    'Status': 'Unknown', 'Future': 'Future'
}
no_spills['Status'] = no_spills['Status'].replace(status_mapping)
print(no_spills['Status'].unique())


flowlineaction_mapping = {
    'Out of Service': 'Out of Service', 'Removed From Service': 'Out of Service',
    'Pre-Abandonment Notice': 'Pre-Abandonment Notice',
    'Abandonment Verification': 'Abandonment Verification',
    'Realignment': 'Realignment',
    'Registration': 'Registration',
    'Abandonment': 'Abandonment'
}
no_spills['FLOWLINEACTION'] = no_spills['FLOWLINEACTION'].replace(flowlineaction_mapping)
print(no_spills['FLOWLINEACTION'].unique())


locationtype_mapping = {
    'Production Facilities': 'Production Facilities', 'Well Site': 'Well Site', 'Manifold': 'Manifold',
    'Compressor Station': 'Compressor Station', 'Gathering Line': 'Gathering Line',
    'Crude Oil Transfer Line': 'Crude Oil Transfer Line', 'Produced Water Transfer System': 'Produced Water Transfer System'
}
no_spills['LOCATIONTYPE'] = no_spills['LOCATIONTYPE'].replace(locationtype_mapping)
print(no_spills['LOCATIONTYPE'].unique())

['Active' 'Out of Service' 'Abandoned' 'Inactive' 'Pending Analysis'
 'Shut In' 'Unknown' 'Future']
['Out of Service' 'Pre-Abandonment Notice' 'Abandonment Verification'
 'Realignment' 'Registration' 'Abandonment']
['Production Facilities' 'Well Site' 'Manifold' 'Compressor Station'
 'Gathering Line' 'Crude Oil Transfer Line'
 'Produced Water Transfer System']


In [15]:
# Update the 'Fluid' column normalization and mapping in one comprehensive block
no_spills['Fluid'] = no_spills['Fluid'].str.strip().str.title().replace({
    'Natual Gas': 'Natural Gas',  # Correct common misspelling
    'Natural Gas Production': 'Natural Gas',
    'Co2': 'Co2/Produced Water',  # Assuming Co2 implies the mixed type
    'C02/Prod Water': 'Co2/Produced Water',
    'Co2/Prod Water': 'Co2/Produced Water',
    'Co2Produced Water': 'Co2/Produced Water',
    'Co2/Produced Wtaer': 'Co2/Produced Water',
    'Gas': 'Natural Gas',  # Assuming general Gas to be Natural Gas
    'Gas, Oil And Water': 'Full Well Stream',
    'Oil': 'Crude Oil',  # Assuming Oil to be categorized as Crude Oil
    'Crude Oil': 'Crude Oil',
    'Crude Oil Emulsion': 'Crude Oil Emulsion',
    'Emulsion': 'Crude Oil Emulsion',  # To unify with Crude Oil Emulsion
    'Crude Oil Emmulsion, Water And Oil': 'Crude Oil Emulsion',
    'Crude Oil And Water Emulsion': 'Crude Oil Emulsion',
    'Oil Water Emulsion': 'Crude Oil Emulsion',
    'Oil/Water': 'Crude Oil Emulsion',
    'Oil /Water/Gas': 'Full Well Stream',
    '3 Phase': 'Multiphase',  # Assumed to mean the same
    'Multiphase': 'Multiphase',
    'Multi-Phase': 'Multiphase',
    'Mulitphase': 'Multiphase',
    'Multi-Phase\xa0': 'Multiphase',  # Non-breaking space issue
    'Injection Produced Water': 'Produced Water',
    'Produced Water': 'Produced Water',
    'Water': 'Produced Water',
    'Saltwater': 'Produced Water',
    'Condensate': 'Condensate',  # Depending on your classification needs
    'Liquid': 'Other',  # General term, assuming to categorize broadly
    'Unprocessed Production Fluids': 'Other',
    'Production Fluids': 'Other',
    'Full Well Stream': 'Full Well Stream',  # Assuming similar handling
    'Other': 'Other',
    'Gas,  Oil And Water' : 'Full Well Stream',
    'Natural Gas Lift': 'Natural Gas',
    'Natuarl Gas': 'Natural Gas',
    'Natural Gas High Pressure': 'Natural Gas',
    'Natural Gas Supply':'Natural Gas',
    'Crude Oill Emulsion': 'Crude Oil Emulsion',
})

# Review the changes to ensure the mappings are applied correctly
print(no_spills['Fluid'].unique())

['Multiphase' 'Crude Oil Emulsion' 'Natural Gas' 'Co2/Produced Water'
 'Crude Oil' 'Produced Water' 'Condensate' 'Other' 'Full Well Stream'
 'Oil Water']


In [16]:
no_spills['Material'] = no_spills['Material'].str.strip().str.title().replace({
    'Fiberglass': 'Fiberglass', 
    'Fibergalss': 'Fiberglass', 
    'Fiberspar': 'Fiberglass', 
    'Fiber Glass': 'Fiberglass',
    'Carbon Steel': 'Carbon Steel', 
    'Carbon Steel - Hdpe': 'Carbon Steel/HDPE', 
    'Carbon Steel, Hdpe,Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel, Hdpe, Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Stainless Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe/Stainless': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe': 'Carbon Steel/HDPE', 
    'Satinless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Steel': 'Steel', 
    'Lined Steel': 'Steel', 
    'Coated Steel': 'Steel', 
    'Flexsteel': 'Steel', 
    'Fiber Glass And Carbon Steel': 'Fiberglass/Carbon Steel', 
    'Fiberglass And Hdpe': 'Fiberglass/HDPE',
    'Hdpe': 'HDPE', 
    'Hdpe Poly': 'HDPE', 
    'Hdpe/Steel': 'HDPE/Steel', 
    'Hdpe Lined Steel': 'HDPE/Steel',
    'Poly': 'Poly', 
    'Polyline': 'Poly', 
    'Poly & Steel': 'Poly/Steel', 
    'Polycarbonate': 'Polycarbonate', 
    'Polycarbonate/Steel': 'Polycarbonate/Steel',
    'Pvc': 'PVC', 
    'Flexspar': 'Fiberglass', 
    'Stainless': 'Steel', 
    'Stainless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Unknown': 'Unknown', 
    'Other': 'Other', 
    'Other (Poly)': 'Other', 
    'Sdr7 Polyethelyne': 'Polyethylene', 
    'Sdr 11 Poly Pipe': 'Polyethylene', 
    'Sdr 11 Poly': 'Polyethylene', 
    'Poly Pipe': 'Polyethylene', 
    'Sdr_Poly': 'Polyethylene',
    'Duplex': 'Duplex', 
    'Fplp': 'Other', 
    'Flowline': 'Other',
    'Flex Steel': 'Steel',
    'Fiberglass And Carbon Steel': 'Fiberglass/Carbon Steel', 
    'Stainless Steel': 'Steel',
    'HDPE Lined Steel': 'HDPE/Steel',
    'Fiberglass/Hdpe': 'Fiberglass/HDPE',
})

# Verify the changes by printing the unique values in the 'Material' column
print(no_spills['Material'].unique())

['Carbon Steel' 'Fiberglass' 'Steel' 'Poly' 'HDPE'
 'Carbon Steel/HDPE/Stainless Steel' 'Carbon Steel/HDPE' 'Unknown' 'Other'
 'Poly/Steel' 'Carbon Steel/Hdpe/Stainless Steel' 'Duplex'
 'Co2/Produced Water' 'PVC' 'Polycarbonate' 'Polycarbonate/Steel'
 'Carbon Steel/Stainless/Hdpe' 'Polyethylene' 'Polypropylene'
 'Fiberglass/Carbon Steel' 'Hdpe/Steel, Flexsteel' 'Fiberglass/HDPE'
 'HDPE/Steel']


In [17]:
no_spills.columns = [
    'operator_number',  # Changed from OPERATOR_NUM to operator_number
    'flowline_id',       # Changed from FLOWLINEID to flowline_id
    'location_id',       # Changed from LOCATION_ID to location_id
    'status',            # Already appropriately named but changed to lowercase
    'flowline_action',   # Changed from FLOWLINEACTION to flowline_action
    'location_type',     # Changed from LOCATIONTYPE to location_type
    'fluid',             # Already appropriately named but changed to lowercase
    'material',          # Already appropriately named but changed to lowercase
    'diameter_in',       # Changed from Diam_in to diameter_in
    'length_ft',         # Changed from Length_ft to length_ft
    'max_operating_pressure', # Changed from MAXOPPRESSURE to max_operating_pressure
    'shape_length',      # Changed from SHAPE_Length to shape_length
    'line_age_yr',
    'geometry',           # Already appropriately named but changed to lowercase
]

# Print new column names to verify
print(no_spills.columns.tolist())

['operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'shape_length', 'line_age_yr', 'geometry']


In [18]:
no_spills.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4117 entries, 1 to 14844
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   operator_number         4117 non-null   int32   
 1   flowline_id             4117 non-null   float64 
 2   location_id             4117 non-null   float64 
 3   status                  4117 non-null   object  
 4   flowline_action         4117 non-null   object  
 5   location_type           4117 non-null   object  
 6   fluid                   4117 non-null   object  
 7   material                4117 non-null   object  
 8   diameter_in             4117 non-null   float64 
 9   length_ft               4117 non-null   float64 
 10  max_operating_pressure  4117 non-null   float64 
 11  shape_length            4117 non-null   float64 
 12  line_age_yr             4117 non-null   float64 
 13  geometry                4117 non-null   geometry
dtypes: float64(7), geome

In [19]:
# make variables integers
no_spills['diameter_in'] = no_spills['diameter_in'].astype(int)
no_spills['length_ft'] = no_spills['length_ft'].astype(int)
no_spills['max_operating_pressure'] = no_spills['max_operating_pressure'].astype(int)
no_spills['shape_length'] = no_spills['shape_length'].astype(int)
no_spills['line_age_yr'] = no_spills['line_age_yr'].astype(int)

In [20]:
# Create a new column 'Risk' in the no_spills DataFrame and set all its values to 0
no_spills['risk'] = 0

## Spills

In [21]:
print(spills.info())
print(spills.shape)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ACTIONDESCRIPTION         5 non-null      object        
 1   BEDDINGMATERIAL           72 non-null     object        
 2   COMPANY_NAME              84 non-null     object        
 3   CONSTRUCTDATE             84 non-null     datetime64[ms]
 4   Detailed Root Cause Type  72 non-null     object        
 5   Diam_in                   84 non-null     float64       
 6   ENDLAT                    84 non-null     float64       
 7   ENDLONG                   84 non-null     float64       
 8   ENTIRELINEREMOVED         0 non-null      object        
 9   FLOWLINEACTION            59 non-null     object        
 10  FLOWLINEID                84 non-null     float64       
 11  Fluid                     84 non-null     object        
 12  LOCATIONTYPE    

In [22]:
spills.describe()

,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,FLOWLINEID,LOCATION_ID,Lat,Length_ft,Long,MAXOPPRESSURE,OPERATOR_NUM,RECEIVE_DATE,SHAPE_Length,STARTLAT,STARTLOCATIONID,STARTLONG,trkg_num
count,84,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,44.000000,84.000000,84,84.000000,84.000000,83.000000,84.000000,8.400000e+01
mean,1995-12-27 01:42:51.428000,4.352679,40.136655,-104.497537,472271.666667,389837.392857,40.140161,21823.340714,-104.489816,179.704545,39992.571429,2020-04-27 18:56:13.063000,6650.083840,40.136851,361573.253012,-104.498610,4.023819e+08
min,1955-04-29 00:00:00,0.000000,37.106749,-108.062538,461137.000000,310063.000000,37.093239,107.770000,-108.063630,10.000000,8960.000000,2018-10-30 17:04:03.327000,32.834947,37.101221,307094.000000,-108.058653,4.017021e+08
25%,1976-10-31 18:00:00,2.000000,40.017682,-104.915835,465833.750000,317985.000000,40.017332,1118.775000,-104.914936,26.000000,10633.000000,2019-09-11 11:29:16.396000,340.945969,40.019395,318185.000000,-104.916080,4.019914e+08
50%,2001-04-17 12:00:00,3.000000,40.113755,-104.705825,474953.500000,429122.000000,40.113877,2166.010000,-104.705578,125.000000,46290.000000,2020-02-14 03:43:34.268000,659.962512,40.112995,328413.000000,-104.702946,4.022982e+08
75%,2014-02-12 18:00:00,6.000000,40.386026,-103.892872,476107.500000,439468.750000,40.385394,6605.540000,-103.810861,150.000000,46290.000000,2020-11-18 15:10:47.893000,2012.563421,40.391077,430308.000000,-103.895688,4.029230e+08
max,2018-02-24 00:00:00,12.750000,40.851863,-102.083250,482303.000000,480689.000000,40.859583,141290.450000,-102.083057,1800.000000,100322.000000,2023-10-13 16:08:29.477000,43053.037992,40.853758,482187.000000,-102.080785,4.032349e+08
std,NaN,3.056949,0.524106,1.048493,5647.787007,66230.664847,0.530457,46037.498444,1.053401,304.851056,31923.495573,NaN,14028.568590,0.524043,55986.941226,1.048925,4.539768e+05


### Get line age from construction date

In [23]:
# Verify CONSTRUCTDATE is datetime type; convert if necessary
spills['CONSTRUCTDATE'] = pd.to_datetime(spills['CONSTRUCTDATE'])

# Calculate line_age
# Today's date
today = pd.Timestamp.now()

# Calculate the difference in years
spills['line_age_yr'] = (today - spills['CONSTRUCTDATE']).dt.days / 365.25

# Display the updated GeoDataFrame to verify 'line_age' column
print(spills[['CONSTRUCTDATE', 'line_age_yr']])

   CONSTRUCTDATE  line_age_yr
0     2014-09-13     9.995893
1     2014-09-13     9.995893
2     2002-10-01    21.946612
3     2018-02-24     6.546201
4     2018-02-24     6.546201
..           ...          ...
79    1975-12-27    48.709103
80    1976-06-29    48.202601
81    1976-08-23    48.052019
82    2008-11-24    15.797399
83    1977-04-15    47.408624

[84 rows x 2 columns]


In [24]:
# Print the list of column names
column_names = spills.columns.tolist()

print(column_names)  

['ACTIONDESCRIPTION', 'BEDDINGMATERIAL', 'COMPANY_NAME', 'CONSTRUCTDATE', 'Detailed Root Cause Type', 'Diam_in', 'ENDLAT', 'ENDLONG', 'ENTIRELINEREMOVED', 'FLOWLINEACTION', 'FLOWLINEID', 'Fluid', 'LOCATIONTYPE', 'LOCATION_ID', 'Lat', 'Length_ft', 'Long', 'MAXOPPRESSURE', 'Material', 'Metallic?', 'OPERATOR_NUM', 'Operator', 'Operator Name', 'PIPEMATERIAL', 'Preventative Measure', 'RECEIVE_DATE', 'Root Cause', 'Root Cause Type', 'SHAPE_Length', 'STARTLAT', 'STARTLOCATIONID', 'STARTLONG', 'Spill Type', 'Spill_Desc', 'Status', 'TYPEOFFLUIDTRANS', 'facility_status', 'facility_type', 'trkg_num', 'geometry', 'line_age_yr']


In [25]:
spills.rename(columns={'OPERATOR_NUM': 'operator_number'}, inplace=True)
spills.rename(columns={'COMPANY_NAME': 'operator_name'}, inplace=True)

# Extract unique OPERATOR_NUM values
unique_operator_nums = spills['operator_number'].unique()

# Create a DataFrame to map OPERATOR_NUM to Operator (assuming the column name for operator names is 'Operator')
spills_operator_mapping = spills[['operator_number', 'operator_name']].drop_duplicates().reset_index(drop=True)

# Apply strip() to remove leading and trailing spaces, and rstrip('.') to remove trailing dots from the 'operator_name' column
spills_operator_mapping['operator_name'] = spills_operator_mapping['operator_name'].str.strip().str.rstrip('.')

# Display the mapping
print(spills_operator_mapping)

    operator_number                               operator_name
0             96155               WHITING OIL & GAS CORPORATION
1             10112            FOUNDATION ENERGY MANAGEMENT LLC
2             10690                       IMPETRO RESOURCES LLC
3             10633       CRESTONE PEAK RESOURCES OPERATING LLC
4             10699                 OWN RESOURCES OPERATING LLC
5             10433                          LARAMIE ENERGY LLC
6             46290                     KP KAUFFMAN COMPANY INC
7             10110         GREAT WESTERN OPERATING COMPANY LLC
8             10598      SANDRIDGE EXPLORATION & PRODUCTION LLC
9            100322                            NOBLE ENERGY INC
10             8960  BONANZA CREEK ENERGY OPERATING COMPANY LLC
11            95520                         WESCO OPERATING INC
12            10672                  TIMBER CREEK OPERATING LLC
13            10706                              D90 ENERGY LLC
14            95620                   WE

In [26]:
# List of columns to be removed
columns_to_remove = [
    "ACTIONDESCRIPTION", "BEDDINGMATERIAL", "operator_name", "CONSTRUCTDATE", "ENDLAT",
    "ENDLONG", "ENTIRELINEREMOVED", "Lat", "Long","Operator","Operator Name", "Spill Type",
    "STARTLAT", "STARTLOCATIONID", "STARTLONG", "trkg_num", "Root Cause", "Detailed Root Cause Type", "Metallic?", 
    "Preventative Measure","Spill_Desc", "TYPEOFFLUIDTRANS", "facility_status", "facility_type", "PIPEMATERIAL", "RECEIVE_DATE"
]

# Drop the columns from the DataFrame
spills = spills.drop(columns=columns_to_remove)

In [27]:
# Print the list of column names
column_names = spills.columns.tolist()

print(column_names)  

['Diam_in', 'FLOWLINEACTION', 'FLOWLINEID', 'Fluid', 'LOCATIONTYPE', 'LOCATION_ID', 'Length_ft', 'MAXOPPRESSURE', 'Material', 'operator_number', 'Root Cause Type', 'SHAPE_Length', 'Status', 'geometry', 'line_age_yr']


In [28]:
# Reorder df
new_order = ['operator_number', 'FLOWLINEID', 'LOCATION_ID', 'Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 
             'Fluid', 'Material', 'Diam_in', 'Length_ft', 'MAXOPPRESSURE', 'SHAPE_Length', 'line_age_yr',
              'Root Cause Type', 'geometry'] 
spills = spills[new_order]
spills

,operator_number,FLOWLINEID,LOCATION_ID,Status,FLOWLINEACTION,LOCATIONTYPE,Fluid,Material,Diam_in,Length_ft,MAXOPPRESSURE,SHAPE_Length,line_age_yr,Root Cause Type,geometry
0,96155,465021.0,435421.0,Active,None,Production Facilities,Crude Oil,Carbon Steel,12.375,141290.45,150.0,43053.037992,9.995893,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956..."
1,96155,465021.0,435421.0,Active,None,Production Facilities,Crude Oil,Carbon Steel,12.375,141290.45,150.0,43053.037992,9.995893,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956..."
2,10112,468013.0,467691.0,Active,None,Production Facilities,Multiphase,Carbon Steel,3.000,1967.95,100.0,599.638869,21.946612,Corrosion,"MULTILINESTRING ((579396.131 4497273.237, 5793..."
3,96155,465449.0,430314.0,Active,Registration,Production Facilities,Natural Gas,Carbon Steel,6.625,129999.02,150.0,39612.461236,6.546201,Unknown,"MULTILINESTRING ((593633.376 4516002.016, 5936..."
4,96155,465449.0,430314.0,Active,Registration,Production Facilities,Natural Gas,Carbon Steel,6.625,129999.02,150.0,39612.461236,6.546201,Incorrect Operation,"MULTILINESTRING ((593633.376 4516002.016, 5936..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,52530,470594.0,451015.0,Active,Registration,Well Site,"Oil, Gas, Water",Poly,2.000,524.75,NaN,159.878478,48.709103,Excavation Damage,"MULTILINESTRING ((495179.357 4469147.98, 49513..."
80,46290,476027.0,318226.0,Active,Registration,Production Facilities,Multiphase,Fiberglass,3.000,433.61,14.0,132.113052,48.202601,Unknown,"MULTILINESTRING ((511486 4440071.988, 511401.6..."
81,46290,478688.0,317631.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,12.750,6311.02,14.0,1922.833152,48.052019,Unknown,"MULTILINESTRING ((509695.403 4437513.863, 5096..."
82,10699,472933.0,337427.0,ACTIVE,Registration,Production Facilities,Natural Gas,HDPE,2.000,1115.25,NaN,340.053258,15.797399,"Pipe, Weld, Joint Failure","MULTILINESTRING ((749596.141 4419849.037, 7495..."


In [29]:
spills.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   operator_number  84 non-null     int32   
 1   FLOWLINEID       84 non-null     float64 
 2   LOCATION_ID      84 non-null     float64 
 3   Status           84 non-null     object  
 4   FLOWLINEACTION   59 non-null     object  
 5   LOCATIONTYPE     84 non-null     object  
 6   Fluid            84 non-null     object  
 7   Material         84 non-null     object  
 8   Diam_in          84 non-null     float64 
 9   Length_ft        84 non-null     float64 
 10  MAXOPPRESSURE    44 non-null     float64 
 11  SHAPE_Length     84 non-null     float64 
 12  line_age_yr      84 non-null     float64 
 13  Root Cause Type  84 non-null     object  
 14  geometry         84 non-null     geometry
dtypes: float64(7), geometry(1), int32(1), object(6)
memory usage: 9.6+ KB


### Consolidate variables uniformly

In [30]:
# List of specific columns for which to print unique values
columns_to_check = ['Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid', 'Material', 'Root Cause Type']

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through the specified columns and store unique values in the dictionary
for column in columns_to_check:
    if column in spills.columns:  # Check if the column exists in the DataFrame
        unique_values = spills[column].unique().tolist()  # Convert numpy array to list
        unique_values_dict[column] = unique_values
    else:
        unique_values_dict[column] = "Column not found in DataFrame."

# Print each column's unique values list
for column, values in unique_values_dict.items():
    print(f"Unique values in {column}: {values}")

Unique values in Status: ['Active', 'ACTIVE', 'Out Of Service', 'Out of Service', 'abandoned']
Unique values in FLOWLINEACTION: [None, 'Registration', 'Out of Service', 'Pre-Abandonment Notice', 'Abandonment Verification']
Unique values in LOCATIONTYPE: ['Production Facilities', 'Gathering Line', 'Manifold', 'Well Site', 'Compressor Station']
Unique values in Fluid: ['Crude Oil', 'Multiphase', 'Natural Gas', 'Crude Oil Emulsion', 'Produced Water', 'PRODUCED WATER', 'Oil', 'Liquid', 'Natural Gas Production', 'Multi-Phase\xa0', 'Oil and water', '3 Phase', 'Oil, Gas, Water']
Unique values in Material: ['Carbon Steel', 'HDPE', 'LINED STEEL', 'Fiberglass', 'Steel', 'HDPE Poly', 'Other (Poly)', 'Other', 'FPLP', 'steel', 'Composite HDPE', 'PVC', 'Poly']
Unique values in Root Cause Type: ['Corrosion', 'Unknown', 'Incorrect Operation', 'Equipment Failure', 'Other Outside Force Damage', 'Natural Force Damage', 'Pipe, Weld, or Joint Failure', 'Excavation Damage', 'Other Outside Force', 'Pipe, Wel

In [31]:
# Define the mapping dictionary for the 'Status' column
status_mapping = {
    'ACTIVE': 'Active',           # Normalize to 'Active'
    'Active': 'Active',           # No change needed, already in desired format
    'Out Of Service': 'Out of Service', # Normalize case and spacing
    'Out of Service': 'Out of Service', # No change needed
    'abandoned': 'Abandoned'      # Capitalize to 'Abandoned'
}

# Apply the mapping to the 'Status' column
spills['Status'] = spills['Status'].replace(status_mapping)
print(spills['Status'].unique())

# Define the mapping dictionary for the 'FLOWLINEACTION' column
flowlineaction_mapping = {
    'Registration': 'Registration',  # No change needed
    None: 'Unknown',                # Mapping None to 'Unknown' (or you could leave it as None if preferred)
    'Abandonment': 'Abandonment',   # No change needed
    'Out of Service': 'Out of Service', # Ensure consistent formatting if there's case variation elsewhere
    'Realignment': 'Realignment',   # No change needed
    'Pre-Abandonment Notice': 'Pre-Abandonment Notice' # No change needed
}

# Apply the mapping to the 'FLOWLINEACTION' column
spills['FLOWLINEACTION'] = spills['FLOWLINEACTION'].replace(flowlineaction_mapping)
print(spills['FLOWLINEACTION'].unique())

['Active' 'Out of Service' 'Abandoned']
['Unknown' 'Registration' 'Out of Service' 'Pre-Abandonment Notice'
 'Abandonment Verification']


In [32]:
# Define the mapping dictionary for the 'Fluid' column, aligning with the provided categories
fluid_mapping = {
    'Crude Oil': 'Crude Oil',                         
    'Multiphase': 'Multiphase',                       
    'Natural Gas': 'Natural Gas',                     
    'Crude Oil Emulsion': 'Crude Oil Emulsion',       
    'Produced Water': 'Produced Water',               
    'PRODUCED WATER': 'Produced Water',               
    'Oil': 'Crude Oil',                               # Mapping general oil as Crude Oil
    'Liquid': 'Other',                                # Mapping generic liquids to 'Other'
    'Natural Gas Production': 'Natural Gas',          # Simplifying to 'Natural Gas'
    'Multi-Phase\xa0': 'Multiphase',                  # Standardizing 'Multiphase'
    'Mulitphase': 'Multiphase',                       # Standardizing 'Multiphase'
    'Oil and water': 'Oil Water',                     # Standardizing as 'Oil Water'
    '3 Phase': 'Multiphase',                          # Mapping to 'Multiphase'
    'Oil, Gas, Water': 'Oil Water',                   # Assuming mix of oil and water primarily
    'Mulitphase': 'Multiphase',                       # Correcting typo and standardizing
    'Co2/Produced Water': 'Co2/Produced Water',       # No change needed, already a standard category
    'Condensate': 'Condensate',                       # Adding as its own category
    'Full Well Stream': 'Full Well Stream'            # Adding as its own category
}

# Apply the mapping to the 'Fluid' column
spills['Fluid'] = spills['Fluid'].replace(fluid_mapping)
print(spills['Fluid'].unique())

['Crude Oil' 'Multiphase' 'Natural Gas' 'Crude Oil Emulsion'
 'Produced Water' 'Other' 'Oil Water']


In [33]:
# Define the mapping dictionary for the 'Material' column
material_mapping = {
    'Carbon Steel': 'Carbon Steel', 
    'HDPE': 'HDPE', 
    'LINED STEEL': 'Steel',          # Assuming Lined Steel is generally categorized as Steel
    'Fiberglass': 'Fiberglass', 
    'Steel': 'Steel', 
    'HDPE Poly': 'HDPE',             # Standardizing as HDPE
    'Other (Poly)': 'Other',         # Simplifying to 'Other'
    'Other': 'Other', 
    'FPLP': 'Other',                 # FPLP is often a specific type of lining or pipe, classify as Other if not specifically relevant
    'steel': 'Steel',                # Ensuring case consistency
    'Composite HDPE': 'HDPE',        # Composite materials containing HDPE, categorized as HDPE
    'PVC': 'PVC', 
    'Poly': 'Poly', 
    'Poly/Steel': 'Poly/Steel',      # This is already in the reference list
    'Carbon Steel/HDPE/Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/HDPE': 'Carbon Steel/HDPE', 
    'Unknown': 'Unknown',            # Keep as is if applicable
    'Duplex': 'Duplex',              # Duplex typically refers to stainless steel but keeping separate as may be specific
    'Co2/Produced Water': 'Co2/Produced Water',   # Specific type of material handling, categorized separately
    'Polycarbonate': 'Polycarbonate',
    'Polycarbonate/Steel': 'Polycarbonate/Steel',
    'Polyethylene': 'Polyethylene',  # Mapping to Polyethylene where applicable
    'Polypropylene': 'Polypropylene',
    'Fiberglass/Carbon Steel': 'Fiberglass/Carbon Steel',
    'Hdpe/Steel, Flexsteel': 'HDPE/Steel'         # Assuming Hdpe/Steel includes any combination like Flexsteel
}

# Apply the mapping to the 'Material' column
spills['Material'] = spills['Material'].replace(material_mapping)
print(spills['Material'].unique())

['Carbon Steel' 'HDPE' 'Steel' 'Fiberglass' 'Other' 'PVC' 'Poly']


In [34]:
# Define the mapping dictionary for the 'Root Cause Type' column
root_cause_mapping = {
    'Corrosion': 'Corrosion', 
    'Unknown': 'Unknown', 
    'Incorrect Operation': 'Incorrect Operation', 
    'Equipment Failure': 'Equipment Failure', 
    'Other Outside Force Damage': 'Other Outside Force Damage', 
    'Natural Force Damage': 'Natural Force Damage', 
    'Pipe, Weld, or Joint Failure': 'Pipe, Weld, or Joint Failure', 
    'Excavation Damage': 'Excavation Damage', 
    'Other Outside Force': 'Other Outside Force Damage',  # Mapping to a more standardized category
    'Pipe, Weld, Joint Failure': 'Pipe, Weld, or Joint Failure'  # Ensuring consistency in naming
}

# Apply the mapping to the 'Root Cause Type' column
spills['Root Cause Type'] = spills['Root Cause Type'].replace(root_cause_mapping)
print(spills['Root Cause Type'].unique())

['Corrosion' 'Unknown' 'Incorrect Operation' 'Equipment Failure'
 'Other Outside Force Damage' 'Natural Force Damage'
 'Pipe, Weld, or Joint Failure' 'Excavation Damage']


In [35]:
# Assuming 'spills' is your DataFrame
spills.columns = [
    'operator_number',         # Changed from OPERATOR_NUM to operator_number
    'flowline_id',             # Changed from FLOWLINEID to flowline_id
    'location_id',             # Changed from LOCATION_ID to location_id
    'status',                  # Already appropriately named but changed to lowercase
    'flowline_action',         # Changed from FLOWLINEACTION to flowline_action
    'location_type',           # Changed from LOCATIONTYPE to location_type
    'fluid',                   # Already appropriately named but changed to lowercase
    'material',                # Already appropriately named but changed to lowercase
    'diameter_in',             # Changed from Diam_in to diameter_in
    'length_ft',               # Changed from Length_ft to length_ft
    'max_operating_pressure',  # Changed from MAXOPPRESSURE to max_operating_pressure
    'shape_length',            # Changed from SHAPE_Length to shape_length
    'line_age_yr',             # Changed from line_age_yr to maintain consistency in the style
    'root_cause_type',         # Added to align with the column names provided earlier
    'geometry'                 # Already appropriately named but changed to lowercase
]

# Print new column names to verify
print(spills.columns.tolist())

['operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'shape_length', 'line_age_yr', 'root_cause_type', 'geometry']


In [36]:
spills.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   operator_number         84 non-null     int32   
 1   flowline_id             84 non-null     float64 
 2   location_id             84 non-null     float64 
 3   status                  84 non-null     object  
 4   flowline_action         84 non-null     object  
 5   location_type           84 non-null     object  
 6   fluid                   84 non-null     object  
 7   material                84 non-null     object  
 8   diameter_in             84 non-null     float64 
 9   length_ft               84 non-null     float64 
 10  max_operating_pressure  44 non-null     float64 
 11  shape_length            84 non-null     float64 
 12  line_age_yr             84 non-null     float64 
 13  root_cause_type         84 non-null     object  
 14  geometry            

In [37]:
# drop any NAs
spills = spills.dropna()
spills.shape

(44, 15)

In [38]:
# make variables integers
spills['diameter_in'] = spills['diameter_in'].astype(int)
spills['length_ft'] = spills['length_ft'].astype(int)
spills['max_operating_pressure'] = spills['max_operating_pressure'].astype(int)
spills['shape_length'] = spills['shape_length'].astype(int)
spills['line_age_yr'] = spills['line_age_yr'].astype(int)

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodatafr

In [39]:
spills['risk'] = 1

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [40]:
spills.head()

,operator_number,flowline_id,location_id,status,flowline_action,location_type,fluid,material,diameter_in,length_ft,max_operating_pressure,shape_length,line_age_yr,root_cause_type,geometry,risk
0,96155,465021.0,435421.0,Active,Unknown,Production Facilities,Crude Oil,Carbon Steel,12,141290,150,43053,9,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956...",1
1,96155,465021.0,435421.0,Active,Unknown,Production Facilities,Crude Oil,Carbon Steel,12,141290,150,43053,9,Corrosion,"MULTILINESTRING ((595672.003 4517988.031, 5956...",1
2,10112,468013.0,467691.0,Active,Unknown,Production Facilities,Multiphase,Carbon Steel,3,1967,100,599,21,Corrosion,"MULTILINESTRING ((579396.131 4497273.237, 5793...",1
3,96155,465449.0,430314.0,Active,Registration,Production Facilities,Natural Gas,Carbon Steel,6,129999,150,39612,6,Unknown,"MULTILINESTRING ((593633.376 4516002.016, 5936...",1
4,96155,465449.0,430314.0,Active,Registration,Production Facilities,Natural Gas,Carbon Steel,6,129999,150,39612,6,Incorrect Operation,"MULTILINESTRING ((593633.376 4516002.016, 5936...",1


# Merge Operator Numer and Name Mapping

In [41]:
# Merge the DataFrames
operator_mapping = pd.merge(no_spills_operator_mapping, spills_operator_mapping, how='outer')

# Drop duplicate rows
operator_mapping.drop_duplicates(inplace=True)
operator_mapping.shape

(120, 2)

In [42]:
print(f"No Spills: {len(no_spills)}")
print(f"Spills: {len(spills)}")

No Spills: 4117
Spills: 44


# Download Cleaned Data

In [43]:
no_spills.to_file("no_spills_cleaned.geojson", driver='GeoJSON')

In [44]:
spills.to_file("spills_cleaned.geojson", driver='GeoJSON')

In [45]:
operator_mapping.to_csv('operator_mapping.csv', index=False)